In [ ]:
import git
import pandas as pd
import datetime

pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv").to_csv("time_series_covid19_confirmed_US.csv", index=False)
pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv").to_csv("time_series_covid19_confirmed_global.csv", index=False)
pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv").to_csv("time_series_covid19_deaths_US.csv", index=False)
pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv").to_csv("time_series_covid19_deaths_global.csv", index=False)
pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv").to_csv("time_series_covid19_recovered_global.csv", index=False)

data_confirmed = pd.read_csv("time_series_covid19_confirmed_global.csv")
data_recovered = pd.read_csv("time_series_covid19_recovered_global.csv")
data_death     = pd.read_csv("time_series_covid19_deaths_global.csv")


data_salida_confirmed = pd.DataFrame() 
data_salida_recovered = pd.DataFrame()
data_salida_death = pd.DataFrame()
data_salida_confirmed["Province/State"] = data_confirmed["Province/State"] 
data_salida_confirmed["Country/Region"] = data_confirmed["Country/Region"] 
data_salida_confirmed["Lat"] = data_confirmed["Lat"]
data_salida_confirmed["Long"] = data_confirmed["Long"]

data_salida_recovered["Country/Region"] = data_recovered["Country/Region"] 
data_salida_recovered["Province/State"] = data_recovered["Province/State"] 
data_salida_death["Country/Region"] = data_death["Country/Region"]

columna_anterior = ""
for columna in data_confirmed.columns[4:]:
    if(columna_anterior == ""):
        #print("no se hace nada")
        pass
    else:
        #print(columna + " y " + columna_anterior)
        try:
            data_salida_confirmed[columna] = data_confirmed[columna] - data_confirmed[columna_anterior]
            data_salida_recovered[columna] = data_recovered[columna] - data_recovered[columna_anterior]
            data_salida_death[columna]     = data_death[columna]     - data_death[columna_anterior]
        except:
            pass
    columna_anterior = columna
    
entrada = {"Province/State":""}   #,"Country/Region":"","Lat":"","Long":""}
salida = []
#index = 0
#data_salida = pd.DataFrame()

data_salida = pd.DataFrame(columns=data_salida_confirmed.columns[:4])

lista_confirmed = list(data_salida_confirmed.iterrows())
lista_recovered = list(data_salida_recovered.iterrows())
lista_death = list(data_salida_death.iterrows())

n = 0

for i in range(len(lista_confirmed)):
    dias_correlativo = 1
    flag = True
    for columna in data_salida_confirmed.columns[4:]:         
        #print(columna)
        #print(i[1][columna])
        entrada["Province/State"] = lista_confirmed[i][1]["Province/State"]
        entrada["Country/Region"] = lista_confirmed[i][1]["Country/Region"]
       
        entrada["Lat"] = lista_confirmed[i][1]["Lat"]
        entrada["Long"] = lista_confirmed[i][1]["Long"]
        formato = columna.split("/")
        entrada["fecha"] = datetime.date(int(formato[2]+"20"),int(formato[0]),int(formato[1])).strftime("%d-%m-%Y")
      
        entrada["confirmados"] = lista_confirmed[i][1][columna]
        entrada["fallecidos"] = lista_death[i][1][columna]
        entrada["codigo"] = entrada["Country/Region"] + str(entrada["Province/State"]) + entrada["fecha"]
        
        if(entrada["confirmados"] != 0):
            print(entrada["Country/Region"] + str(dias_correlativo))
            dias_correlativo += 1
        else:
            print(entrada["Country/Region"] + "0")
            
        
        #entrada["recuperados"] = lista_recovered[i][1][columna]
        salida.append(data_salida.append(entrada,ignore_index=True))
data_salida = pd.concat(salida)

entrada = {"Province/State":""}
salida = []
data_salida_aux = pd.DataFrame(columns=data_salida_confirmed.columns[:4])
for i in range(len(lista_recovered)):
    for columna in data_salida_recovered.columns[4:]:
        entrada["Province/State"] = lista_recovered[i][1]["Province/State"]
        entrada["Country/Region"] = lista_recovered[i][1]["Country/Region"]
        formato = columna.split("/")
        entrada["fecha"] = datetime.date(int(formato[2]+"20"),int(formato[0]),int(formato[1])).strftime("%d-%m-%Y")
        entrada["recuperado"] = lista_recovered[i][1][columna]
        entrada["codigo"] = entrada["Country/Region"] + str(entrada["Province/State"]) + entrada["fecha"]
        #print(entrada)
        #print("*****************************************************************")
        salida.append(data_salida_aux.append(entrada, ignore_index=True))
data_salida_aux = pd.concat(salida)
del data_salida_aux["Province/State"]
del data_salida_aux["Country/Region"]
del data_salida_aux["fecha"]
del data_salida_aux["Lat"]
del data_salida_aux["Long"]


merged_left = pd.merge(left=data_salida, right=data_salida_aux, how='left', left_on='codigo', right_on='codigo')
#merged_left


#merged_left.to_csv("acumuladoLuis.csv", index=False)
merged_left.to_csv("acumulado.csv", index=False)


repoLocal = git.Repo( 'C:/Users/limc_/Documents/GitHub/Datos' )
print(repoLocal.git.status())
repoLocal.git.add(".")
try:
    repoLocal.git.commit(m='Update automatico via python')
except:
    pass

origin = repoLocal.remote(name='origin')
origin.push()

Afghanistan0
Afghanistan0
Afghanistan0
Afghanistan0
Afghanistan0
Afghanistan0
Afghanistan0
Afghanistan0
Afghanistan0
Afghanistan0
Afghanistan0
Afghanistan0
Afghanistan0
Afghanistan0
Afghanistan0
Afghanistan0
Afghanistan0
Afghanistan0
Afghanistan0
Afghanistan0
Afghanistan0
Afghanistan0
Afghanistan0
Afghanistan0
Afghanistan0
Afghanistan0
Afghanistan0
Afghanistan0
Afghanistan0
Afghanistan0
Afghanistan0
Afghanistan0
Afghanistan1
Afghanistan0
Afghanistan0
Afghanistan0
Afghanistan0
Afghanistan0
Afghanistan0
Afghanistan0
Afghanistan0
Afghanistan0
Afghanistan0
Afghanistan0
Afghanistan0
Afghanistan2
Afghanistan0
Afghanistan3
Afghanistan4
Afghanistan0
Afghanistan0
Afghanistan5
Afghanistan6
Afghanistan7
Afghanistan8
Afghanistan0
Afghanistan0
Afghanistan9
Afghanistan0
Afghanistan10
Afghanistan0
Afghanistan11
Afghanistan12
Afghanistan13
Afghanistan14
Afghanistan0
Afghanistan15
Afghanistan16
Afghanistan17
Afghanistan18
Albania0
Albania0
Albania0
Albania0
Albania0
Albania0
Albania0
Albania0
Albania0


Australia0
Australia2
Australia0
Australia0
Australia0
Australia3
Australia0
Australia0
Australia0
Australia0
Australia0
Australia0
Australia0
Australia0
Australia0
Australia4
Australia0
Australia5
Australia0
Australia6
Australia0
Australia7
Australia0
Australia8
Australia0
Australia0
Australia9
Australia10
Australia0
Australia0
Australia0
Australia0
Australia0
Australia0
Australia1
Australia2
Australia3
Australia4
Australia5
Australia0
Australia6
Australia0
Australia7
Australia8
Australia0
Australia0
Australia0
Australia0
Australia0
Australia0
Australia0
Australia0
Australia0
Australia0
Australia0
Australia0
Australia0
Australia0
Australia0
Australia0
Australia0
Australia0
Australia0
Australia0
Australia0
Australia9
Australia0
Australia0
Australia10
Australia0
Australia11
Australia0
Australia0
Australia12
Australia0
Australia13
Australia14
Australia0
Australia15
Australia16
Australia17
Australia18
Australia19
Australia20
Australia21
Australia22
Australia23
Australia24
Australia25
Aust

Barbados0
Barbados0
Barbados0
Barbados0
Barbados0
Barbados0
Barbados0
Barbados1
Barbados0
Barbados2
Barbados0
Barbados3
Barbados4
Barbados5
Barbados6
Barbados0
Barbados0
Barbados7
Barbados8
Barbados9
Barbados0
Barbados10
Barbados0
Belarus0
Belarus0
Belarus0
Belarus0
Belarus0
Belarus0
Belarus0
Belarus0
Belarus0
Belarus0
Belarus0
Belarus0
Belarus0
Belarus0
Belarus0
Belarus0
Belarus0
Belarus0
Belarus0
Belarus0
Belarus0
Belarus0
Belarus0
Belarus0
Belarus0
Belarus0
Belarus0
Belarus0
Belarus0
Belarus0
Belarus0
Belarus0
Belarus0
Belarus0
Belarus0
Belarus0
Belarus1
Belarus0
Belarus0
Belarus0
Belarus0
Belarus2
Belarus0
Belarus0
Belarus0
Belarus0
Belarus0
Belarus3
Belarus0
Belarus4
Belarus5
Belarus0
Belarus0
Belarus6
Belarus0
Belarus7
Belarus0
Belarus8
Belarus9
Belarus0
Belarus10
Belarus0
Belarus11
Belarus0
Belarus12
Belarus0
Belarus0
Belarus13
Belarus0
Belarus14
Belgium0
Belgium0
Belgium0
Belgium0
Belgium0
Belgium0
Belgium0
Belgium0
Belgium0
Belgium0
Belgium0
Belgium0
Belgium1
Belgium0
Belgium0

Cabo Verde0
Cabo Verde0
Cabo Verde0
Cabo Verde3
Cabo Verde0
Cabo Verde4
Cabo Verde0
Cabo Verde5
Cabo Verde0
Cabo Verde0
Cabo Verde0
Cambodia0
Cambodia0
Cambodia0
Cambodia0
Cambodia1
Cambodia0
Cambodia0
Cambodia0
Cambodia0
Cambodia0
Cambodia0
Cambodia0
Cambodia0
Cambodia0
Cambodia0
Cambodia0
Cambodia0
Cambodia0
Cambodia0
Cambodia0
Cambodia0
Cambodia0
Cambodia0
Cambodia0
Cambodia0
Cambodia0
Cambodia0
Cambodia0
Cambodia0
Cambodia0
Cambodia0
Cambodia0
Cambodia0
Cambodia0
Cambodia0
Cambodia0
Cambodia0
Cambodia0
Cambodia0
Cambodia0
Cambodia0
Cambodia0
Cambodia0
Cambodia0
Cambodia0
Cambodia2
Cambodia0
Cambodia0
Cambodia3
Cambodia0
Cambodia4
Cambodia5
Cambodia0
Cambodia0
Cambodia6
Cambodia7
Cambodia8
Cambodia9
Cambodia10
Cambodia11
Cambodia12
Cambodia13
Cambodia14
Cambodia0
Cambodia15
Cambodia0
Cambodia16
Cambodia17
Cambodia18
Cambodia0
Cameroon0
Cameroon0
Cameroon0
Cameroon0
Cameroon0
Cameroon0
Cameroon0
Cameroon0
Cameroon0
Cameroon0
Cameroon0
Cameroon0
Cameroon0
Cameroon0
Cameroon0
Cameroon0

Central African Republic0
Central African Republic0
Central African Republic0
Central African Republic0
Central African Republic0
Central African Republic0
Central African Republic0
Central African Republic0
Central African Republic0
Central African Republic0
Central African Republic0
Central African Republic0
Central African Republic0
Central African Republic0
Central African Republic0
Central African Republic0
Central African Republic0
Central African Republic0
Central African Republic0
Central African Republic0
Central African Republic0
Central African Republic0
Central African Republic0
Central African Republic0
Central African Republic0
Central African Republic0
Central African Republic0
Central African Republic1
Central African Republic0
Central African Republic0
Central African Republic0
Central African Republic0
Central African Republic2
Central African Republic0
Central African Republic0
Central African Republic0
Central African Republic0
Central African Republic0
Central Afri

...

Seychelles0
Seychelles0
Seychelles0
Seychelles0
Seychelles0
Seychelles0
Seychelles0
Seychelles0
Seychelles0
Seychelles0
Seychelles0
Seychelles0
Seychelles0
Seychelles0
Seychelles0
Seychelles0
Seychelles0
Seychelles0
Seychelles0
Seychelles0
Seychelles0
Seychelles0
Seychelles0
Seychelles0
Seychelles0
Seychelles0
Seychelles0
Seychelles0
Seychelles0
Seychelles1
Seychelles0
Seychelles2
Seychelles3
Seychelles0
Seychelles4
Seychelles5
Seychelles0
Seychelles0
Seychelles0
Seychelles0
Seychelles0
Seychelles0
Seychelles0
Seychelles6
Seychelles0
Seychelles0
Seychelles7
Seychelles0
Singapore1
Singapore2
Singapore0
Singapore3
Singapore4
Singapore5
Singapore0
Singapore6
Singapore7
Singapore8
Singapore9
Singapore0
Singapore10
Singapore11
Singapore0
Singapore12
Singapore13
Singapore14
Singapore15
Singapore16
Singapore17
Singapore18
Singapore19
Singapore20
Singapore21
Singapore22
Singapore23
Singapore24
Singapore0
Singapore25
Singapore0
Singapore26
Singapore0
Singapore27
Singapore28
Singapore0
Singapore

Taiwan*0
Taiwan*2
Taiwan*3
Taiwan*4
Taiwan*0
Taiwan*5
Taiwan*6
Taiwan*0
Taiwan*0
Taiwan*0
Taiwan*7
Taiwan*0
Taiwan*8
Taiwan*0
Taiwan*9
Taiwan*10
Taiwan*0
Taiwan*0
Taiwan*0
Taiwan*0
Taiwan*0
Taiwan*0
Taiwan*11
Taiwan*12
Taiwan*0
Taiwan*13
Taiwan*14
Taiwan*15
Taiwan*0
Taiwan*16
Taiwan*17
Taiwan*18
Taiwan*19
Taiwan*0
Taiwan*20
Taiwan*21
Taiwan*22
Taiwan*23
Taiwan*24
Taiwan*0
Taiwan*25
Taiwan*26
Taiwan*0
Taiwan*0
Taiwan*0
Taiwan*27
Taiwan*28
Taiwan*29
Taiwan*30
Taiwan*31
Taiwan*32
Taiwan*33
Taiwan*34
Taiwan*35
Taiwan*36
Taiwan*37
Taiwan*38
Taiwan*39
Taiwan*40
Taiwan*41
Taiwan*42
Taiwan*43
Taiwan*44
Taiwan*45
Taiwan*46
Taiwan*47
Taiwan*48
Taiwan*49
Tanzania0
Tanzania0
Tanzania0
Tanzania0
Tanzania0
Tanzania0
Tanzania0
Tanzania0
Tanzania0
Tanzania0
Tanzania0
Tanzania0
Tanzania0
Tanzania0
Tanzania0
Tanzania0
Tanzania0
Tanzania0
Tanzania0
Tanzania0
Tanzania0
Tanzania0
Tanzania0
Tanzania0
Tanzania0
Tanzania0
Tanzania0
Tanzania0
Tanzania0
Tanzania0
Tanzania0
Tanzania0
Tanzania0
Tanzania0
Tanzania

United Kingdom0
United Kingdom0
United Kingdom0
United Kingdom0
United Kingdom0
United Kingdom0
United Kingdom0
United Kingdom0
United Kingdom0
United Kingdom0
United Kingdom0
United Kingdom0
United Kingdom0
United Kingdom0
United Kingdom0
United Kingdom0
United Kingdom0
United Kingdom0
United Kingdom0
United Kingdom0
United Kingdom0
United Kingdom0
United Kingdom0
United Kingdom0
United Kingdom0
United Kingdom0
United Kingdom0
United Kingdom0
United Kingdom0
United Kingdom0
United Kingdom0
United Kingdom0
United Kingdom0
United Kingdom0
United Kingdom0
United Kingdom0
United Kingdom0
United Kingdom1
United Kingdom0
United Kingdom0
United Kingdom2
United Kingdom0
United Kingdom0
United Kingdom3
United Kingdom4
United Kingdom5
United Kingdom0
United Kingdom6
United Kingdom7
United Kingdom8
United Kingdom0
United Kingdom0
United Kingdom0
United Kingdom0
United Kingdom0
United Kingdom0
United Kingdom0
United Kingdom0
United Kingdom0
United Kingdom0
United Kingdom0
United Kingdom0
United K

US0
US0
US0
US3
US4
US0
US5
US0
US0
US0
US0
US0
US0
US0
US6
US0
US7
US0
US0
US0
US0
US0
US0
US0
US8
US0
US0
US9
US0
US10
US11
US12
US13
US14
US15
US16
US17
US18
US19
US20
US21
US22
US23
US24
US25
US26
US27
US28
US29
US30
US31
US32
US33
US34
US35
US36
US37
US38
US39
US40
US41
US42
US43
US44
US45
Uzbekistan0
Uzbekistan0
Uzbekistan0
Uzbekistan0
Uzbekistan0
Uzbekistan0
Uzbekistan0
Uzbekistan0
Uzbekistan0
Uzbekistan0
Uzbekistan0
Uzbekistan0
Uzbekistan0
Uzbekistan0
Uzbekistan0
Uzbekistan0
Uzbekistan0
Uzbekistan0
Uzbekistan0
Uzbekistan0
Uzbekistan0
Uzbekistan0
Uzbekistan0
Uzbekistan0
Uzbekistan0
Uzbekistan0
Uzbekistan0
Uzbekistan0
Uzbekistan0
Uzbekistan0
Uzbekistan0
Uzbekistan0
Uzbekistan0
Uzbekistan0
Uzbekistan0
Uzbekistan0
Uzbekistan0
Uzbekistan0
Uzbekistan0
Uzbekistan0
Uzbekistan0
Uzbekistan0
Uzbekistan0
Uzbekistan0
Uzbekistan0
Uzbekistan0
Uzbekistan0
Uzbekistan0
Uzbekistan0
Uzbekistan0
Uzbekistan0
Uzbekistan0
Uzbekistan1
Uzbekistan2
Uzbekistan3
Uzbekistan4
Uzbekistan5
Uzbekistan6
Uzbekist

Belize0
Belize0
Belize0
Belize0
Belize0
Belize0
Belize0
Belize0
Belize0
Belize0
Belize0
Belize0
Belize0
Belize0
Belize0
Belize0
Belize0
Belize0
Belize0
Belize0
Belize0
Belize0
Belize0
Belize0
Belize0
Belize0
Belize0
Belize0
Belize0
Belize0
Belize0
Belize1
Belize0
Belize2
Belize0
Belize0
Belize0
Belize0
Belize3
Belize0
Belize0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Canada0
Laos0
Laos0
Laos0
Laos0
Laos0
Laos0
Laos0
Laos0
Laos0
Laos0
Laos0
Laos0
Laos0
Laos0
Laos0
Laos0
Laos0
Laos0
Laos

{'Province/State': nan, 'Country/Region': 'Afghanistan', 'fecha': '26-01-2020', 'recuperado': 0, 'codigo': 'Afghanistannan26-01-2020'}
*****************************************************************
{'Province/State': nan, 'Country/Region': 'Afghanistan', 'fecha': '27-01-2020', 'recuperado': 0, 'codigo': 'Afghanistannan27-01-2020'}
*****************************************************************
{'Province/State': nan, 'Country/Region': 'Afghanistan', 'fecha': '28-01-2020', 'recuperado': 0, 'codigo': 'Afghanistannan28-01-2020'}
*****************************************************************
{'Province/State': nan, 'Country/Region': 'Afghanistan', 'fecha': '29-01-2020', 'recuperado': 0, 'codigo': 'Afghanistannan29-01-2020'}
*****************************************************************
{'Province/State': nan, 'Country/Region': 'Afghanistan', 'fecha': '30-01-2020', 'recuperado': 0, 'codigo': 'Afghanistannan30-01-2020'}
*************************************************************

*****************************************************************
{'Province/State': nan, 'Country/Region': 'Afghanistan', 'fecha': '12-03-2020', 'recuperado': 0, 'codigo': 'Afghanistannan12-03-2020'}
*****************************************************************
{'Province/State': nan, 'Country/Region': 'Afghanistan', 'fecha': '13-03-2020', 'recuperado': 0, 'codigo': 'Afghanistannan13-03-2020'}
*****************************************************************
{'Province/State': nan, 'Country/Region': 'Afghanistan', 'fecha': '14-03-2020', 'recuperado': 0, 'codigo': 'Afghanistannan14-03-2020'}
*****************************************************************
{'Province/State': nan, 'Country/Region': 'Afghanistan', 'fecha': '15-03-2020', 'recuperado': 0, 'codigo': 'Afghanistannan15-03-2020'}
*****************************************************************
{'Province/State': nan, 'Country/Region': 'Afghanistan', 'fecha': '16-03-2020', 'recuperado': 1, 'codigo': 'Afghanistannan16-03-20

{'Province/State': nan, 'Country/Region': 'Albania', 'fecha': '06-03-2020', 'recuperado': 0, 'codigo': 'Albanianan06-03-2020'}
*****************************************************************
{'Province/State': nan, 'Country/Region': 'Albania', 'fecha': '07-03-2020', 'recuperado': 0, 'codigo': 'Albanianan07-03-2020'}
*****************************************************************
{'Province/State': nan, 'Country/Region': 'Albania', 'fecha': '08-03-2020', 'recuperado': 0, 'codigo': 'Albanianan08-03-2020'}
*****************************************************************
{'Province/State': nan, 'Country/Region': 'Albania', 'fecha': '09-03-2020', 'recuperado': 0, 'codigo': 'Albanianan09-03-2020'}
*****************************************************************
{'Province/State': nan, 'Country/Region': 'Albania', 'fecha': '10-03-2020', 'recuperado': 0, 'codigo': 'Albanianan10-03-2020'}
*****************************************************************
{'Province/State': nan, 'Country/Re

{'Province/State': nan, 'Country/Region': 'Algeria', 'fecha': '05-03-2020', 'recuperado': 0, 'codigo': 'Algerianan05-03-2020'}
*****************************************************************
{'Province/State': nan, 'Country/Region': 'Algeria', 'fecha': '06-03-2020', 'recuperado': 0, 'codigo': 'Algerianan06-03-2020'}
*****************************************************************
{'Province/State': nan, 'Country/Region': 'Algeria', 'fecha': '07-03-2020', 'recuperado': 0, 'codigo': 'Algerianan07-03-2020'}
*****************************************************************
{'Province/State': nan, 'Country/Region': 'Algeria', 'fecha': '08-03-2020', 'recuperado': 0, 'codigo': 'Algerianan08-03-2020'}
*****************************************************************
{'Province/State': nan, 'Country/Region': 'Algeria', 'fecha': '09-03-2020', 'recuperado': 0, 'codigo': 'Algerianan09-03-2020'}
*****************************************************************
{'Province/State': nan, 'Country/Re

{'Province/State': nan, 'Country/Region': 'Andorra', 'fecha': '16-02-2020', 'recuperado': 0, 'codigo': 'Andorranan16-02-2020'}
*****************************************************************
{'Province/State': nan, 'Country/Region': 'Andorra', 'fecha': '17-02-2020', 'recuperado': 0, 'codigo': 'Andorranan17-02-2020'}
*****************************************************************
{'Province/State': nan, 'Country/Region': 'Andorra', 'fecha': '18-02-2020', 'recuperado': 0, 'codigo': 'Andorranan18-02-2020'}
*****************************************************************
{'Province/State': nan, 'Country/Region': 'Andorra', 'fecha': '19-02-2020', 'recuperado': 0, 'codigo': 'Andorranan19-02-2020'}
*****************************************************************
{'Province/State': nan, 'Country/Region': 'Andorra', 'fecha': '20-02-2020', 'recuperado': 0, 'codigo': 'Andorranan20-02-2020'}
*****************************************************************
{'Province/State': nan, 'Country/Re

...

{'Province/State': 'Turks and Caicos Islands', 'Country/Region': 'United Kingdom', 'fecha': '02-02-2020', 'recuperado': 0, 'codigo': 'United KingdomTurks and Caicos Islands02-02-2020'}
*****************************************************************
{'Province/State': 'Turks and Caicos Islands', 'Country/Region': 'United Kingdom', 'fecha': '03-02-2020', 'recuperado': 0, 'codigo': 'United KingdomTurks and Caicos Islands03-02-2020'}
*****************************************************************
{'Province/State': 'Turks and Caicos Islands', 'Country/Region': 'United Kingdom', 'fecha': '04-02-2020', 'recuperado': 0, 'codigo': 'United KingdomTurks and Caicos Islands04-02-2020'}
*****************************************************************
{'Province/State': 'Turks and Caicos Islands', 'Country/Region': 'United Kingdom', 'fecha': '05-02-2020', 'recuperado': 0, 'codigo': 'United KingdomTurks and Caicos Islands05-02-2020'}
**************************************************************

{'Province/State': 'Turks and Caicos Islands', 'Country/Region': 'United Kingdom', 'fecha': '16-03-2020', 'recuperado': 0, 'codigo': 'United KingdomTurks and Caicos Islands16-03-2020'}
*****************************************************************
{'Province/State': 'Turks and Caicos Islands', 'Country/Region': 'United Kingdom', 'fecha': '17-03-2020', 'recuperado': 0, 'codigo': 'United KingdomTurks and Caicos Islands17-03-2020'}
*****************************************************************
{'Province/State': 'Turks and Caicos Islands', 'Country/Region': 'United Kingdom', 'fecha': '18-03-2020', 'recuperado': 0, 'codigo': 'United KingdomTurks and Caicos Islands18-03-2020'}
*****************************************************************
{'Province/State': 'Turks and Caicos Islands', 'Country/Region': 'United Kingdom', 'fecha': '19-03-2020', 'recuperado': 0, 'codigo': 'United KingdomTurks and Caicos Islands19-03-2020'}
**************************************************************

{'Province/State': nan, 'Country/Region': 'MS Zaandam', 'fecha': '29-03-2020', 'recuperado': 0, 'codigo': 'MS Zaandamnan29-03-2020'}
*****************************************************************
{'Province/State': nan, 'Country/Region': 'MS Zaandam', 'fecha': '30-03-2020', 'recuperado': 0, 'codigo': 'MS Zaandamnan30-03-2020'}
*****************************************************************
{'Province/State': nan, 'Country/Region': 'MS Zaandam', 'fecha': '31-03-2020', 'recuperado': 0, 'codigo': 'MS Zaandamnan31-03-2020'}
*****************************************************************
{'Province/State': nan, 'Country/Region': 'Botswana', 'fecha': '26-01-2020', 'recuperado': 0, 'codigo': 'Botswananan26-01-2020'}
*****************************************************************
{'Province/State': nan, 'Country/Region': 'Botswana', 'fecha': '27-01-2020', 'recuperado': 0, 'codigo': 'Botswananan27-01-2020'}
*****************************************************************
{'Province/St

{'Province/State': nan, 'Country/Region': 'Burundi', 'fecha': '30-01-2020', 'recuperado': 0, 'codigo': 'Burundinan30-01-2020'}
*****************************************************************
{'Province/State': nan, 'Country/Region': 'Burundi', 'fecha': '31-01-2020', 'recuperado': 0, 'codigo': 'Burundinan31-01-2020'}
*****************************************************************
{'Province/State': nan, 'Country/Region': 'Burundi', 'fecha': '01-02-2020', 'recuperado': 0, 'codigo': 'Burundinan01-02-2020'}
*****************************************************************
{'Province/State': nan, 'Country/Region': 'Burundi', 'fecha': '02-02-2020', 'recuperado': 0, 'codigo': 'Burundinan02-02-2020'}
*****************************************************************
{'Province/State': nan, 'Country/Region': 'Burundi', 'fecha': '03-02-2020', 'recuperado': 0, 'codigo': 'Burundinan03-02-2020'}
*****************************************************************
{'Province/State': nan, 'Country/Re

{'Province/State': nan, 'Country/Region': 'Sierra Leone', 'fecha': '08-02-2020', 'recuperado': 0, 'codigo': 'Sierra Leonenan08-02-2020'}
*****************************************************************
{'Province/State': nan, 'Country/Region': 'Sierra Leone', 'fecha': '09-02-2020', 'recuperado': 0, 'codigo': 'Sierra Leonenan09-02-2020'}
*****************************************************************
{'Province/State': nan, 'Country/Region': 'Sierra Leone', 'fecha': '10-02-2020', 'recuperado': 0, 'codigo': 'Sierra Leonenan10-02-2020'}
*****************************************************************
{'Province/State': nan, 'Country/Region': 'Sierra Leone', 'fecha': '11-02-2020', 'recuperado': 0, 'codigo': 'Sierra Leonenan11-02-2020'}
*****************************************************************
{'Province/State': nan, 'Country/Region': 'Sierra Leone', 'fecha': '12-02-2020', 'recuperado': 0, 'codigo': 'Sierra Leonenan12-02-2020'}
***************************************************

In [44]:
#print(data_salida_recovered[data_salida_recovered["Country/Region"] == "Chile"])
#data_salida_recovered[data_salida_recovered["Country/Region"] == "Chile"] 
#print(data_recovered[data_recovered["Country/Region"] == "Chile"])
#data_recovered
merged_left[merged_left["Country/Region"] == "Chile"].head()

   Country/Region  1/24/20  1/25/20  1/26/20  1/27/20  1/28/20  1/29/20  \
39          Chile        0        0        0        0        0        0   

    1/30/20  1/31/20  2/1/20  ...  3/22/20  3/23/20  3/24/20  3/25/20  \
39        0        0       0  ...        2        0        9        5   

    3/26/20  3/27/20  3/28/20  3/29/20  3/30/20  3/31/20  
39        0       21       18       14       81        0  

[1 rows x 69 columns]
   Province/State Country/Region      Lat    Long  1/22/20  1/23/20  1/24/20  \
39            NaN          Chile -35.6751 -71.543        0        0        0   

    1/25/20  1/26/20  1/27/20  ...  3/22/20  3/23/20  3/24/20  3/25/20  \
39        0        0        0  ...        8        8       17       22   

    3/26/20  3/27/20  3/28/20  3/29/20  3/30/20  3/31/20  
39       22       43       61       75      156      156  

[1 rows x 74 columns]


,Province/State,Country/Region,Lat,Long,codigo,confirmados,fallecidos,fecha,recuperado
3327,NaN,Chile,-35.6751,-71.543,Chilenan27-03-2020,304.0,1.0,27-03-2020,0.0
3328,NaN,Chile,-35.6751,-71.543,Chilenan28-03-2020,299.0,1.0,28-03-2020,0.0
3329,NaN,Chile,-35.6751,-71.543,Chilenan29-03-2020,230.0,1.0,29-03-2020,0.0
3330,NaN,Chile,-35.6751,-71.543,Chilenan30-03-2020,310.0,1.0,30-03-2020,0.0
3331,NaN,Chile,-35.6751,-71.543,Chilenan31-03-2020,289.0,4.0,31-03-2020,0.0


In [33]:
data_salida.shape, data_salida_aux.shape, merged_left.shape

((17408, 8), (15730, 2), (17408, 9))

In [46]:
data_salida_aux[data_salida_aux["codigo"] == "Chilenan30-03-2020"]

,codigo,recuperado
0,Chilenan30-03-2020,0.0


In [51]:
lista_recovered[0][1]

Country/Region    Afghanistan
1/24/20                     0
1/25/20                     0
1/26/20                     0
1/27/20                     0
                     ...     
3/27/20                     0
3/28/20                     0
3/29/20                     0
3/30/20                     0
3/31/20                     3
Name: 0, Length: 69, dtype: object

In [54]:
data_confirmed.columns

Index(['Province/State', 'Country/Region', 'Lat', 'Long', '1/22/20', '1/23/20',
       '1/24/20', '1/25/20', '1/26/20', '1/27/20', '1/28/20', '1/29/20',
       '1/30/20', '1/31/20', '2/1/20', '2/2/20', '2/3/20', '2/4/20', '2/5/20',
       '2/6/20', '2/7/20', '2/8/20', '2/9/20', '2/10/20', '2/11/20', '2/12/20',
       '2/13/20', '2/14/20', '2/15/20', '2/16/20', '2/17/20', '2/18/20',
       '2/19/20', '2/20/20', '2/21/20', '2/22/20', '2/23/20', '2/24/20',
       '2/25/20', '2/26/20', '2/27/20', '2/28/20', '2/29/20', '3/1/20',
       '3/2/20', '3/3/20', '3/4/20', '3/5/20', '3/6/20', '3/7/20', '3/8/20',
       '3/9/20', '3/10/20', '3/11/20', '3/12/20', '3/13/20', '3/14/20',
       '3/15/20', '3/16/20', '3/17/20', '3/18/20', '3/19/20', '3/20/20',
       '3/21/20', '3/22/20', '3/23/20', '3/24/20', '3/25/20', '3/26/20',
       '3/27/20', '3/28/20', '3/29/20', '3/30/20', '3/31/20'],
      dtype='object')

In [60]:
type(lista_confirmed[1][1]["3/13/20"])

int